In [1]:
import pandas as pd
import numpy as np
import pulp as plp

from IPython.display import display, Math, Latex

In [2]:
#importing Data

In [3]:
flights = pd.read_excel("../WiDS/flight_data.xlsx", sheet_name="flights")
fleet = pd.read_excel("../WiDS/flight_data.xlsx", sheet_name="fleet")
timezone = pd.read_excel("../WiDS/flight_data.xlsx", sheet_name="timeZone")

In [4]:
# Math Model

Variables: 
$$x_{t,f,f^\prime} =  \left\{ 
\begin{array}{l}
1 \text{ if flight } f^\prime \text{ is covered after flight } f \text{ by aircraft } t\\
0 \text{ otherwise}
\end{array}
\right.$$

Constraints:

1 - Every flight must be allocated to an aircraft:
$$
\sum_{t\in T}\sum_{f^\prime \in F} x_{t,f,f^\prime} = 1,\qquad \forall f \in F
$$

2 - We cannot used more than available aircrafts:

This constraints is taking into consideration while building the variables $x_{t,f,f^\prime}$ : we don't built a variable for for a non existing aircraft

3 - Minimum ground Time (MGT):

After covering the flight $f$ the aircraft $t$ must have a MGT of 45 min before flying $f^\prime$. This constraints is taking into consideration while building the variables $x_{t,f,f^\prime}$ : there is at least 45 min between the arrival time of $f$ and the departure time of $f^\prime$.


4 - Non-broken paths for aicrafts:

$$
\sum_{f^\prime\in F} x_{t,f^\prime,f} = \sum_{f^\prime \in F} x_{t,f,f^\prime} ,\qquad \forall f \in F, \forall t\in T
$$

5 - The plan of an aircraft must start \& end in DXB:

One trick to implement this is to add for every aircraft two "virtual" flights departing \& landing in DXB.

In [5]:
# converting times to UTC
timezone = timezone.set_index('airport').to_dict('index')
from datetime import timedelta
flights['dptTime'] = flights.apply(
    lambda x: x['dptTime'] + timedelta(hours=timezone[x['org']]['to_utc']),
    axis=1
)

flights['arrTime'] = flights.apply(
    lambda x: x['arrTime'] + timedelta(hours=timezone[x['dst']]['to_utc']),
    axis=1
)

In [6]:
# keeping a copy of the data
solution = flights.copy(deep=True)

In [7]:
flights["virtual"] = False

In [8]:
from pandas import Timestamp

virtual_1 = {
    'ID' : 0,
    'Flight Number' : 'XX9001',
    'org' : 'DXB',
    'dst' : 'DXB',
    'dptTime' : Timestamp('2019-01-05 00:00:00'),
    'arrTime' : Timestamp('2019-01-05 00:00:00'),
    'subfleet' : 'Boeing 737-800',
    'virtual' : True
}
virtual_2 = {
    'ID' : 100,
    'Flight Number' : 'XX9002',
    'org' : 'DXB',
    'dst' : 'DXB',
    'dptTime' : Timestamp('2019-01-07 00:00:00'),
    'arrTime' : Timestamp('2019-01-07 00:00:00'),
    'subfleet': 'Boeing 737-800',
    'virtual' : True
}



In [9]:
flights = flights.append(virtual_1, ignore_index=True)
flights = flights.append(virtual_2, ignore_index=True)

In [10]:
# creating variables

In [11]:
def create_var(flight_ID_1, flight_ID_2, aircraft_ID, list_variables):
        list_variables.append({
            'var': plp.LpVariable(
                cat= plp.LpBinary,
                name= aircraft_ID\
                    + "_flt_" + np.str(flight_ID_1)\
                    + "_" + np.str(flight_ID_2)
            ),
            'aircraft': aircraft_ID,
            'flt1': flight_ID_1,
            'flt2': flight_ID_2
        
        })
        
        return list_variables
        

In [12]:
def vars_for_next_flts(flight_ID, dst, arrTime, df_nextFlts, df_fleet):
    from datetime import timedelta
    mgt = 45 #min
    timestamp = arrTime + timedelta(minutes=mgt)
    df = df_nextFlts.loc[df_nextFlts.ID != flight_ID]
    df = df.loc[df.org == dst]
    df = df.loc[df.dptTime >= timestamp]

    list_variables = []
    for ac in fleet.aircraft.unique():
        for value in df.values:
            next_flight = value[0]
            list_variables = create_var(flight_ID, next_flight, ac, list_variables)

    return list_variables

In [13]:
flights["variables"] = flights.apply(
    lambda x: vars_for_next_flts(
        x['ID'], x['dst'], x['arrTime'], flights.copy(deep=True), fleet
    ), axis=1
             )


In [14]:
# flight cover constraints

In [15]:
flights['flight_cover'] = flights.apply(
    lambda x: 
    plp.LpConstraint(
        e=plp.lpSum(y['var'] for y in x['variables']),
        sense=plp.LpConstraintEQ,
        rhs= 1,
        name= "cover_flt_" + np.str(x['ID'])
    ) if (x['virtual'] == False) else None, 
    axis=1
)


In [16]:
# path constraints

In [17]:
flights['in_variables'] = flights.apply(
    lambda x: dict.fromkeys(fleet.aircraft.unique()), axis=1)
flights['out_variables'] = flights.apply(
    lambda x: dict.fromkeys(fleet.aircraft.unique()), axis=1)

In [18]:
def find_in_vars(flightID, dic_in_variables, df_flights):
    for ac in dic_in_variables:
        dic_in_variables[ac] = []
    for list_variables in df_flights.variables:
        for var in list_variables:
            if (var['flt2'] == flightID):
                dic_in_variables[var['aircraft']].append(var)


In [19]:
def find_out_vars(flightID, dic_out_variables, df_flights):
    for ac in dic_out_variables:
        dic_out_variables[ac] = []
    for list_variables in df_flights.variables:
        for var in list_variables:
            if (var['flt1'] == flightID):
                dic_out_variables[var['aircraft']].append(var)

In [20]:
status = flights.apply(
    lambda x: find_in_vars(x['ID'], x['in_variables'], flights.copy(deep=True)),
    axis=1
)

In [21]:
status = flights.apply(
    lambda x: find_out_vars(x['ID'], x['out_variables'], flights.copy(deep=True)),
    axis=1
)

In [22]:
def create_path_constraints(flightID, virtual, df_flights, df_fleet):
    dict_path_constraints = dict.fromkeys(fleet.aircraft.unique())
    in_variables = df_flights.loc[df_flights.ID ==flightID]['in_variables'].values[0]
    out_variables = df_flights.loc[df_flights.ID ==flightID]['out_variables'].values[0]
    
    for ac in dict_path_constraints:
        dict_path_constraints[ac] = plp.LpConstraint(
            e=plp.lpSum(variable['var'] for variable in in_variables[ac])- plp.lpSum(variable['var'] for variable in out_variables[ac]),
            sense=plp.LpConstraintEQ,
            rhs=0 if virtual == False else (-1 if flightID ==0 else +1),
            name="path_" + ac + "_" + np.str(flightID)
        )
    return dict_path_constraints

In [23]:
flights['path_cntr'] = flights.apply(
    lambda x: create_path_constraints(
        x['ID'], x['virtual'], flights.copy(deep=True), fleet),
    axis=1
)

In [24]:
opt_model = plp.LpProblem(name="aircraft_ass_model")

In [25]:
# add varaibles
status = flights.apply( 
    lambda x: 
    [opt_model.addVariable(y['var']) for y in x['variables']],
    axis=1
)

In [26]:
# add constraints
status = flights.loc[flights.virtual == False].apply( 
    lambda x: opt_model.addConstraint(x['flight_cover']),axis=1
)

In [27]:
status = flights.apply( 
    lambda x: [opt_model.addConstraint(x['path_cntr'][ac]) for ac in x['path_cntr']],axis=1
)

In [28]:
opt_model.writeLP("aircraft_ass_model.LP")

In [29]:
status = opt_model.solve()
if(status == 1):
    print("an optimal solution has been found")
else:
    print("no solution found")

an optimal solution has been found


In [30]:
# processing data

In [31]:
varsdict = {}
for v in opt_model.variables():
    if v.name == '__dummy': continue
    varsdict[v.name] = v.varValue

In [32]:
dict_flight_ass = {}
for v in varsdict:
    if varsdict[v] > 0:
        var_split = v.split("_")
        if var_split[2] == '0':
            continue
        dict_flight_ass[var_split[2]] = var_split[0]

In [33]:
solution['aircraft'] = solution.apply(lambda x: dict_flight_ass[np.str(x['ID'])], axis=1)

In [34]:
solution = flights.copy(deep=True)

In [35]:
# writing solution

In [36]:
solution.to_csv("aicraft_allocations.csv")